### 3D CNN with tensorflow

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D # for 3d plotting

import h5py

%matplotlib inline

# Input data files are available in the {path} directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
# print(check_output(["dir", data_path]).decode("utf8"))

# Any results you write to the current directory are saved as output.

#### Load data

In [3]:
import sys
import os
if sys.platform == 'darwin':
    data_path = os.getcwd() + "/3d-mnist/full_dataset_vectors.h5"
else:
    data_path = os.getcwd() + "\\3d-mnist\\full_dataset_vectors.h5"
    
# load the data
with h5py.File(data_path, 'r') as hf:
    x_train_raw = hf["X_train"][:]
    y_train_raw = hf["y_train"][:]
    x_test_raw = hf["X_test"][:]
    y_test_raw = hf["y_test"][:]


# length check
assert(len(x_train_raw) == len(y_train_raw))
assert(len(x_test_raw) == len(y_test_raw))

In [8]:
# 1D vector to rgb values, provided by ../input/plot3d.py
def array_to_color(array, cmap="Oranges"):
    s_m = plt.cm.ScalarMappable(cmap=cmap)
    return s_m.to_rgba(array)[:,:-1]

# Transform data from 1d to 3d rgb
def rgb_data_transform(data):
    data_t = []
    for i in range(data.shape[0]):
        data_t.append(array_to_color(data[i]).reshape(16, 16, 16, 3))
    return np.asarray(data_t, dtype=np.float32)

def to_categorical(label, n_classes):
    label_list = []
    for index, value in enumerate(label):
        x = np.zeros(n_classes)
        x[value] = 1
        label_list.append(x)
    return label_list

In [9]:
n_classes = 10 # from 0 to 9, 10 labels totally

x_train = rgb_data_transform(x_train_raw)
x_test = rgb_data_transform(x_test_raw)

y_train = to_categorical(y_train_raw, n_classes)
y_test = to_categorical(y_test_raw, n_classes)

In [28]:
x_train_raw[1].shape

(4096,)

In [58]:
# %load ./voxel_grid_plot.py
import os
import shutil

import numpy as np

from IPython.display import IFrame
from matplotlib import pyplot as plt


def plot_voxelgrid(voxelgrid,
                   output_name=None,
                   cmap="Oranges",
                   axis=True,
                   width=800,
                   height=600):

    scaled_shape = voxelgrid.shape

    vector = voxelgrid
    points = np.argwhere(vector) * scaled_shape

    s_m = plt.cm.ScalarMappable(cmap=cmap)
    rgb = s_m.to_rgba(vector.reshape(-1)[vector.reshape(-1) > 0])

    camera_position = points.max(0) + abs(points.max(0))

    look = points.mean(0)

    if axis:
        axis_size = points.ptp() * 1.5
    else:
        axis_size = 0

    placeholders = {}

    placeholders["POINTS_X_PLACEHOLDER"] = points[:, 0].tolist()
    placeholders["POINTS_Y_PLACEHOLDER"] = points[:, 1].tolist()
    placeholders["POINTS_Z_PLACEHOLDER"] = points[:, 2].tolist()

    placeholders["R_PLACEHOLDER"] = rgb[:, 0].tolist()
    placeholders["G_PLACEHOLDER"] = rgb[:, 1].tolist()
    placeholders["B_PLACEHOLDER"] = rgb[:, 2].tolist()

    placeholders["S_x_PLACEHOLDER"] = scaled_shape[0]
    placeholders["S_y_PLACEHOLDER"] = scaled_shape[1]
    placeholders["S_z_PLACEHOLDER"] = scaled_shape[2]

    placeholders["CAMERA_X_PLACEHOLDER"] = camera_position[0]
    placeholders["CAMERA_Y_PLACEHOLDER"] = camera_position[1]
    placeholders["CAMERA_Z_PLACEHOLDER"] = camera_position[2]

    placeholders["LOOK_X_PLACEHOLDER"] = look[0]
    placeholders["LOOK_Y_PLACEHOLDER"] = look[1]
    placeholders["LOOK_Z_PLACEHOLDER"] = look[2]

    placeholders["AXIS_SIZE_PLACEHOLDER"] = axis_size

    placeholders["N_VOXELS_PLACEHOLDER"] = sum(vector.reshape(-1) > 0)

    if output_name is None:
        output_name = "plotVG.html"

    BASE_PATH = os.getcwd()
    src = "{}/{}".format(BASE_PATH, "voxelgrid.html")
    dst = "{}/{}".format(os.getcwd(), output_name)

    with open(src, "r") as inp, open(dst, "w") as out:
        for line in inp:
            for key, val in placeholders.items():
                if key in line:
                    line = line.replace(key, str(val))
            out.write(line)

    return IFrame(output_name, width=width, height=height)

In [59]:
plot_voxelgrid(x_train_raw[1].reshape((16,16,16)))

Input placeholder init

In [6]:
with tf.name_scope('inputs'):
    x_input = tf.placeholder(tf.float32, shape=[None, 16, 16, 16, 3])
    y_input = tf.placeholder(tf.float32, shape=[None, n_classes]) 

Construct CNN model with 3 conv layers and apply dropout in final layer.

In [7]:
def cnn_model(x_train_data, keep_rate=0.7, seed=None):
    
    with tf.name_scope("layer_a"):
        # conv => 16*16*16
        conv1 = tf.layers.conv3d(inputs=x_train_data, filters=16, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu)
        # conv => 16*16*16
        conv2 = tf.layers.conv3d(inputs=conv1, filters=32, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu)
        # pool => 8*8*8
        pool3 = tf.layers.max_pooling3d(inputs=conv2, pool_size=[2, 2, 2], strides=2)
        
    with tf.name_scope("layer_c"):
        # conv => 8*8*8
        conv4 = tf.layers.conv3d(inputs=pool3, filters=64, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu)
        # conv => 8*8*8
        conv5 = tf.layers.conv3d(inputs=conv4, filters=128, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu)
        # pool => 4*4*4
        pool6 = tf.layers.max_pooling3d(inputs=conv5, pool_size=[2, 2, 2], strides=2)
        
    with tf.name_scope("batch_norm"):
        cnn3d_bn = tf.layers.batch_normalization(inputs=pool6, training=True)
        
    with tf.name_scope("fully_con"):
        flattening = tf.reshape(cnn3d_bn, [-1, 4*4*4*128])
        dense = tf.layers.dense(inputs=flattening, units=1024, activation=tf.nn.relu)
        # (1-keep_rate) is the probability that the node will be kept
        dropout = tf.layers.dropout(inputs=dense, rate=keep_rate, training=True)
        
    with tf.name_scope("y_conv"):
        y_conv = tf.layers.dense(inputs=dropout, units=10)
    
    return y_conv

In [13]:
def train_neural_network(x_train_data, y_train_data, x_test_data, y_test_data, learning_rate=0.1, keep_rate=0.7, epochs=10, batch_size=128, using_gpu=False):

    if using_gpu:
        device_name = '/gpu:1' 
    else:
        device_name = '/cpu:0'

    with tf.device(device_name):
        with tf.name_scope("cross_entropy"):
            prediction = cnn_model(x_input, keep_rate, seed=1)
            cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y_input))
                              
        with tf.name_scope("training"):
            optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_input, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        
    logs_path = os.path.join(os.getcwd(), '3d_mnist_cnn_tensorboard')

    tf.summary.scalar("cross_entropy", cost)
    tf.summary.scalar("accuracy", accuracy)
    # command line for tensorboard $tensorboard --logdir=train:`logs_path`/train,test:`logs_path`/test --port 6006
    # merge all summaries into a single "operation" which can execute in a session
    summary_all = tf.summary.merge_all()
 
    sess = tf.Session()
    
    train_writer = tf.summary.FileWriter(os.path.join(logs_path, 'train'), graph=tf.get_default_graph())
    test_writer = tf.summary.FileWriter(os.path.join(logs_path, 'test'), graph=tf.get_default_graph())
    
    if using_gpu:
        # GPU using BFC
        config = tf.ConfigProto()
        config.gpu_options.allocator_type = 'BFC'
        sess =  tf.Session(config= config)
            
    sess.run(tf.global_variables_initializer())
    import datetime

    start_time = datetime.datetime.now()

    iterations = int(len(x_train_data)/batch_size) + 1
    # run epochs
    for epoch in range(epochs):
        start_time_epoch = datetime.datetime.now()
        print('Epoch', epoch, 'started', end='')
        epoch_loss = 0
        # mini batch
        for itr in range(iterations):
            mini_batch_x = x_train_data[itr*batch_size: (itr+1)*batch_size]
            mini_batch_y = y_train_data[itr*batch_size: (itr+1)*batch_size]
            _optimizer, _cost = sess.run([optimizer, cost], feed_dict={x_input: mini_batch_x, y_input: mini_batch_y})
            epoch_loss += _cost

        #  using mini batch in case not enough memory
#         acc = 0
        itrs = int(len(x_test_data)/batch_size) + 1
        for itr in range(itrs):
#             mini_batch_x_test = x_test_data[itr*batch_size: (itr+1)*batch_size]
#             mini_batch_y_test = y_test_data[itr*batch_size: (itr+1)*batch_size]
#             acc += sess.run(accuracy, feed_dict={x_input: mini_batch_x_test, y_input: mini_batch_y_test})
            pass

        train_sum = sess.run(summary_all, feed_dict={x_input: x_train_data, y_input: y_train_data})
        acc, test_sum = sess.run([accuracy, summary_all], feed_dict={x_input: x_test_data, y_input: y_test_data})
        
        train_writer.add_summary(train_sum, epoch)
        test_writer.add_summary(test_sum, epoch)
        train_writer.flush()
        test_writer.flush()
        end_time_epoch = datetime.datetime.now()
        print(' Testing Set Accuracy:',acc/itrs, ' Time elapse: ', str(end_time_epoch - start_time_epoch))

    end_time = datetime.datetime.now()
    print('Time elapse: ', str(end_time - start_time))
    sess.close()

In [14]:
train_neural_network(x_train[:10], y_train[:10], x_test[:50], y_test[:50], learning_rate=0.01, batch_size=32, epochs=50, using_gpu=False)

Epoch 0 started Testing Set Accuracy: 0.0299999993294  Time elapse:  0:00:33.097672
Epoch 1 started Testing Set Accuracy: 0.0399999991059  Time elapse:  0:00:33.517426
Epoch 2 started Testing Set Accuracy: 0.070000000298  Time elapse:  0:00:33.420720
Epoch 3 started Testing Set Accuracy: 0.070000000298  Time elapse:  0:00:32.971733
Epoch 4 started Testing Set Accuracy: 0.0900000035763  Time elapse:  0:00:34.016386
Epoch 5 started Testing Set Accuracy: 0.070000000298  Time elapse:  0:00:32.915203
Epoch 6 started Testing Set Accuracy: 0.0799999982119  Time elapse:  0:00:32.911776
Epoch 7 started Testing Set Accuracy: 0.0599999986589  Time elapse:  0:00:33.601286
Epoch 8 started Testing Set Accuracy: 0.070000000298  Time elapse:  0:00:32.936826
Epoch 9 started

KeyboardInterrupt: 

In [ ]:
with tf.Session() as sess:
    prediction = cnn_model(x_input, keep_rate=0.5)
    sess.run(tf.global_variables_initializer())
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_input, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    print('Accuracy:',accuracy.eval({x_input:x_train[:2], y_input:y_train[:2]}))

In [ ]:
fig = plt.figure(figsize=(16,9))
for i in range(16):
    ax = fig.add_subplot(4,4,i+1)
    plt.imshow(x_train[1][i,:,:])

print(y_train[1])